In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import chi2_contingency, ttest_ind, shapiro
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.iolib.summary2 import summary_col
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler  # Ensure this is imported
import os  # For folder creation
import ast  # For safely evaluating string representations of lists

In [10]:
# Define the file path to your dataset
filepath = "D:/daten_masterarbeit/final_dataset_reg_full.csv"

# Read the CSV file
df = pd.read_csv(filepath)

print(f"Number of observations in the final_dataset: {len(df)}")

#%% Data Preparation

# List of variables to include in the analysis
variables = [
    'similarity_to_overall_average',
    'similarity_to_industry_average',
    'similarity_to_company_average',
    'excess_ret_immediate',
    'excess_ret_short_term',
    'excess_ret_medium_term',
    'excess_ret_long_term',
    'epsfxq',
    'epsfxq_next',
    'length_participant_questions',  # Dependent Variable
    'length_management_answers',    # Dependent Variable
    'market_cap',                   # Control Variable
    'rolling_beta',                 # Control Variable
    'ceo_participates',             # Control Variable
    'ceo_cfo_change',               # Control Variable
    'word_length_presentation',     # Control Variable
    'participant_question_topics',  # For Chi-Squared Test
    'management_answer_topics',     # For Chi-Squared Test
    'filtered_presentation_topics'  # For topic diversity
]


Number of observations in the final_dataset: 50073


In [17]:

# Ensure all variables exist in the DataFrame
missing_vars = [var for var in variables if var not in df.columns]
if missing_vars:
    raise KeyError(f"The following required columns are missing from the DataFrame: {missing_vars}")

# Create analysis DataFrame with the specified variables
analysis_df = df[variables].dropna()

# Display the number of observations after dropping NaNs
print(f"Number of observations after dropping NaNs: {len(analysis_df)}")

Number of observations after dropping NaNs: 41500


In [18]:
analysis_df.columns

Index(['similarity_to_overall_average', 'similarity_to_industry_average',
       'similarity_to_company_average', 'excess_ret_immediate',
       'excess_ret_short_term', 'excess_ret_medium_term',
       'excess_ret_long_term', 'epsfxq', 'epsfxq_next',
       'length_participant_questions', 'length_management_answers',
       'market_cap', 'rolling_beta', 'ceo_participates', 'ceo_cfo_change',
       'word_length_presentation', 'participant_question_topics',
       'management_answer_topics', 'filtered_presentation_topics'],
      dtype='object')

In [19]:
analysis_df = analysis_df[['similarity_to_overall_average', 'similarity_to_industry_average', 'similarity_to_company_average','length_management_answers', 'length_participant_questions', 'filtered_presentation_topics']]

In [23]:
import ast
import pandas as pd

# Ensure that 'analysis_df' is your DataFrame
# and that 'filtered_presentation_topics' contains string representations of lists.

# Calculate the 20th and 80th quantiles once to avoid recalculating
quantile_20 = analysis_df['similarity_to_industry_average'].quantile(0.2)
quantile_80 = analysis_df['similarity_to_industry_average'].quantile(0.8)

# Filter the lowest 20% of similarity_to_overall_average
lowest_20_df = analysis_df[analysis_df['similarity_to_industry_average'] <= quantile_20]

# Calculate the average length of filtered_presentation_topics vectors for lowest 20%
lowest_20_avg_length = lowest_20_df['filtered_presentation_topics'].apply(
    lambda x: len(ast.literal_eval(x)) if isinstance(x, str) else len(x)
).mean()

print(f"Average length of filtered_presentation_topics (Lowest 20%): {lowest_20_avg_length}")

# Filter the highest 20% of similarity_to_overall_average
highest_20_df = analysis_df[analysis_df['similarity_to_industry_average'] >= quantile_80]

# Calculate the average length of filtered_presentation_topics vectors for highest 20%
highest_20_avg_length = highest_20_df['filtered_presentation_topics'].apply(
    lambda x: len(ast.literal_eval(x)) if isinstance(x, str) else len(x)
).mean()

print(f"Average length of filtered_presentation_topics (Highest 20%): {highest_20_avg_length}")


Average length of filtered_presentation_topics (Lowest 20%): 149.00506024096384
Average length of filtered_presentation_topics (Highest 20%): 130.65867469879518


explore transition matrices

In [1]:
import pandas as pd

# Path to your transition matrix file
file_path = r"C:\Users\nikla\OneDrive\Dokumente\winfoMaster\Masterarbeit\regression_results\overall_transition_matrix.csv"

# Read the CSV file into a pandas DataFrame
transition_matrix_df = pd.read_csv(file_path, index_col=0)

# Display the DataFrame
transition_matrix_df.head()


,To_0,To_1,To_2,To_3,To_4,To_5,To_6,To_7,To_8,To_9,...,To_80,To_81,To_82,To_83,To_84,To_85,To_86,To_87,To_88,To_89
From_0,7396,907,593,143,1067,2921,433,2279,1662,138,...,163,32,15,27,8,11,52,140,2,19
From_1,4785,663,130,277,23,1536,2628,1079,214,71,...,637,56,7,14,11,4,37,210,8,3
From_2,2395,3129,341,67,19,135,346,188,276,43,...,1315,225,81,46,12,16,151,973,12,19
From_3,556,657,265,75,11,1293,226,134,464,16,...,219,13,9,1,1,1,30,89,2,4
From_4,577,2423,4656,908,35,28,14,44,197,123,...,38,4,5,3,2,2,36,22,3,9


In [8]:
import numpy as np
import pandas as pd

# Assuming `transition_matrix_df` is the loaded DataFrame
# Convert DataFrame to a numpy array for easier processing
transition_matrix = transition_matrix_df.to_numpy()

# Normalize the rows to get probabilities
row_sums = transition_matrix.sum(axis=1, keepdims=True)
probability_matrix = np.divide(
    transition_matrix,
    row_sums,
    where=row_sums != 0  # Avoid division by zero
)

# Convert back to a DataFrame for better exploration
probability_matrix_df = pd.DataFrame(
    probability_matrix,
    index=transition_matrix_df.index,
    columns=transition_matrix_df.columns
)

# Save the normalized matrix to a CSV file for further exploration (optional)
probability_matrix_df.to_csv("normalized_transition_matrix.csv")
print("Normalized transition matrix saved as 'normalized_transition_matrix.csv'.")

# Find the top N most frequent transitions
N = 20  # Adjust N to show more or fewer transitions
flat_indices = np.argsort(transition_matrix, axis=None)[-N:]  # Indices of the top N values in flattened array
top_values = transition_matrix.flatten()[flat_indices]  # Corresponding values of these indices
rows, cols = np.unravel_index(flat_indices, transition_matrix.shape)  # Convert to 2D indices

# Create a DataFrame for better visualization
frequent_transitions = pd.DataFrame({
    "From Topic": rows,
    "To Topic": cols,
    "Frequency": top_values
}).sort_values(by="Frequency", ascending=False)

# Display the top transitions
#print("Top frequent transitions:")
#print(frequent_transitions)

# To view the corresponding probabilities of the top transitions:
frequent_transitions["Probability"] = probability_matrix[rows, cols]
#sort by Probability
frequent_transitions = frequent_transitions.sort_values(by="Probability", ascending=False)
print("Top frequent transitions with probabilities:")
print(frequent_transitions)


Normalized transition matrix saved as 'normalized_transition_matrix.csv'.
Top frequent transitions with probabilities:
    From Topic  To Topic  Frequency  Probability
0           33        33      18877     0.443322
5           69        69      22414     0.352200
6           87        87      22464     0.344856
1            5         4      18886     0.324323
10          72        72      26038     0.297631
19          23        23     135337     0.285963
14          14        12      29782     0.254084
12          40        40      27033     0.251482
4           86        86      22185     0.232621
2           21        21      19343     0.226236
7           25        25      22911     0.219761
9           11        11      25985     0.205311
13          14        14      29161     0.202912
18          19        19      66203     0.184751
8           68        68      24025     0.168747
3           70        70      19405     0.153917
15          78        78      35976     0.148879

In [12]:
len(df["siccd"].unique())

528

In [19]:
df_grouped = df.groupby("siccd")["permco"].nunique()
industries_with_10_permcos = df_grouped[df_grouped >= 5]
count_of_industries = len(industries_with_10_permcos)
print(count_of_industries)


101


In [14]:
df["siccd"].freq()

AttributeError: 'Series' object has no attribute 'freq'

In [27]:
#read this to pandas
#"D:\daten_masterarbeit\CRSP_monthly_Compustat_quarterly_merged.csv" 
acc_daten = pd.read_csv("D:\daten_masterarbeit\CRSP_monthly_Compustat_quarterly_merged.csv", nrows=100000)


<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\nikla\AppData\Local\Temp\ipykernel_31948\1070434396.py:3: SyntaxWarning: invalid escape sequence '\d'
  acc_daten = pd.read_csv("D:\daten_masterarbeit\CRSP_monthly_Compustat_quarterly_merged.csv", nrows=100000)


In [28]:
acc_daten.columns

Index(['permno', 'date', 'exchcd', 'siccd', 'ncusip', 'ticker', 'comnam',
       'permco', 'prc', 'vol', 'ret', 'shrout', 'cfacpr', 'cfacshr',
       'month_id', 'year', 'gvkey', 'month_id_datadate', 'datadate', 'fyearq',
       'fqtr', 'fyr', 'conm', 'datacqtr', 'datafqtr', 'rdq', 'atq', 'ceqq',
       'dlcq', 'dlttq', 'epsfiq', 'epsfxq', 'epspiq', 'epspxq', 'ibq', 'ltq',
       'niq', 'oiadpq', 'xidoq', 'xiq'],
      dtype='object')

In [29]:
acc_daten[["epsfxq"]]

,epsfxq
0,NaN
1,NaN
2,0.59
3,NaN
4,NaN
...,...
99995,-1.22
99996,NaN
99997,NaN
99998,-1.48
